In [17]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from datetime import timedelta
import numpy as np
import random
import json


class WordleGame:
    def __init__(self):
        self.Menu = tk.Tk()
        self.Menu.title("Bootleg Wordle Menu")
        self.Menu.geometry("600x830")

        self.Welcome_Message = tk.Label(self.Menu, text="Welcome to the main menu of Bootleg Wordle! \n"
                                                         "Here, you can select a difficulty, and learn how to play.",
                                       font=('Times New Roman', 17)
                                       )
        self.Welcome_Message.pack()

        self.How_to_play = tk.Label(self.Menu, text="How to play:")
        self.How_to_play.pack(pady=10)

        self.Rules_Message = tk.Label(self.Menu, text="Wordle is a word guessing game.\n"
                                                      "You have 6 guesses to match a randomly chosen word!\n"
                                                      "Each guess must be 5 letters long.\n"
                                                      "The color of the tiles will change to show how close\n"
                                                      "your guess was to the word.",
                                     font=('Times New Roman', 15)
                                     )
        self.Rules_Message.pack(pady=10)

        self.Rules_Message.pack()
        self.difficulty_var = tk.StringVar()

        self.label_difficulty = tk.Label(self.Menu, text="Please select a Difficulty:", font=('Arial', 18))
        self.label_difficulty.pack(pady=10)

        style = ttk.Style()
        style.configure("TRadiobutton", indicatorsize=0)

        self.radio_easy = ttk.Radiobutton(self.Menu, text="Easy", variable=self.difficulty_var, value="Easy",
                                          style="TRadiobutton", padding=10)
        self.radio_easy.pack()

        self.radio_medium = ttk.Radiobutton(self.Menu, text="Medium", variable=self.difficulty_var, value="Medium",
                                            style="TRadiobutton", padding=10)
        self.radio_medium.pack()

        self.radio_hard = ttk.Radiobutton(self.Menu, text="Hard", variable=self.difficulty_var, value="Hard",
                                          style="TRadiobutton", padding=10)
        self.radio_hard.pack()

        self.Start_Game = tk.Button(self.Menu, text="Start Game", command=self.start_game)
        self.Start_Game.pack()

        self.root = None
        self.timer_label = None
        self.turn = 1
        self.actualWord = None

        self.Menu.mainloop()

    def choose_random_word(self):
        file_path = r'C:\Users\Dell\Downloads\words.json'
        with open(file_path, 'r') as file:
            data = json.load(file)
        return random.choice(data).upper()

    def create_game_ui(self, difficulty):
        self.root = tk.Tk()
        self.root.title("Bootleg Wordle")
        self.root.geometry("600x830")

        self.textbox = tk.Text(self.root, height=2, font=('Arial', 18))
        self.textbox.pack(padx=20, pady=5)

        self.label = tk.Label(self.root, text='Please Enter a Guess!', font=('Arial', 18))
        self.label.pack()

        self.buttonframe = tk.Frame(self.root)
        self.buttonframe.columnconfigure(0, weight=1)
        self.buttonframe.columnconfigure(1, weight=1)
        self.buttonframe.columnconfigure(2, weight=1)

        self.Alphabet = "QWERTYUIOPASDFGHJKLZXCVBNM"
        self.buttons = []
        for index, letter in enumerate(self.Alphabet):
            btnltr = tk.Button(self.buttonframe, text=letter, font=('Arial', 18), command=lambda l=letter: self.insert_text(l))
            if 0 <= index <= 9:
                btnltr.grid(row=0, column=index, sticky=tk.W + tk.E)
            elif 10 <= index <= 18:
                btnltr.grid(row=1, column=index - 10, sticky=tk.W + tk.E)
            else:
                btnltr.grid(row=2, column=index - 19, sticky=tk.W + tk.E)
            self.buttons.append(btnltr)

        self.buttonframe.pack()

        self.btnDelete = tk.Button(self.buttonframe, text='Delete', font=('Arial', 18), command=self.Backspace)
        self.btnDelete.grid(row=2, column=7, columnspan=2, sticky=tk.W + tk.E)
        self.buttonframe.pack()

        self.btnfrm2 = tk.Frame(self.root)
        self.btnfrm2.columnconfigure(0, weight=1)
        self.btnfrm2.columnconfigure(1, weight=1)
        self.btnfrm2.columnconfigure(2, weight=1)
        self.btnSubmit = tk.Button(self.btnfrm2, text='Guess', font=('Arial', 18), command=self.actual_game)
        self.btnSubmit.grid()
        self.btnfrm2.pack()

        self.lose_win_length_label = tk.Label(self.root, text="", font=('Arial', 24))
        self.lose_win_length_label.pack()

   
        self.timer_label = tk.Label(self.root, text="", font=('Arial', 18))
        self.timer_label.pack(pady=10)

       
        self.restart_button = tk.Button(self.root, text="Restart Game", command=self.restart_game)
        self.restart_button.pack(pady=10)

    def start_game(self):
        self.Menu.destroy()
        difficulty = self.difficulty_var.get()
        self.actualWord = self.choose_random_word()

        self.create_game_ui(difficulty)

       
        self.timer = TimerApp(self.root, self, difficulty)

        self.root.protocol("WM_DELETE_WINDOW", self.on_window_close)
        self.root.mainloop()

    def on_window_close(self):
        if self.timer:
            self.timer.stop_timer()  
        self.root.destroy()

    def insert_text(self, letter):
        current_text = self.textbox.get("1.0", "end-1c")
        new_text = current_text + letter
        self.textbox.delete("1.0", "end")
        self.textbox.insert("1.0", new_text)

    def Backspace(self):
        self.textbox.delete("end-2c", "end")

    def ClearText(self):
        self.textbox.delete("1.0", "end")

    def actual_game(self):
        guess = self.textbox.get("1.0", "end-1c").upper().strip()
        Black = '#000000'
        Yellow = '#c8b653'
        Green = '#6ca965'
        gylist = [Black, Black, Black, Black, Black]

        if len(guess) != 5:
            self.lose_win_length_label.config(text="Only 5 Letter Words!")
        else:
            self.lose_win_length_label.config(text="")
            if self.turn <= 5:
                for gcNum, guessChar in enumerate(guess):
                    for acNum, acChar in enumerate(self.actualWord):
                        if (guessChar == acChar and gcNum == acNum):
                            gylist[gcNum] = Green

                for acNum, acChar in enumerate(self.actualWord):
                    if gylist[acNum] != Green:
                        for gcNum, guessChar in enumerate(guess):
                            if guessChar == acChar and gylist[gcNum] == Black:
                                gylist[gcNum] = Yellow
                                break

                for i, letter in enumerate(guess):
                    position = self.Alphabet.index(letter)
                    current_color = self.buttons[position].cget("fg")
                    if current_color != Green and current_color != Yellow and gylist[i] != Black:
                        self.buttons[position].config(fg=gylist[i])
                    elif current_color != Green and current_color != Yellow and gylist[i] == Black:
                        self.buttons[position].config(fg='#808080')
                    elif current_color == Yellow and gylist[i] == Green:
                        self.buttons[position].config(fg=gylist[i])

                GuessLabels = tk.Frame(self.root)
                GuessLabels.columnconfigure(3, weight=1)
                GuessLabels.columnconfigure(4, weight=1)
                GuessLabels.columnconfigure(5, weight=1)

                k = 0
                for k in range(0, 5):
                    GuessLetter = tk.Label(GuessLabels, text=guess[k], font=('Arial', 16), bg=gylist[k], width=4,
                                           height=2)
                    GuessLetter.grid(row=self.turn, column=k, sticky=tk.W + tk.E, padx=5, pady=5)
                    GuessLabels.pack()

                if all(i == gylist[0] for i in gylist) and gylist[0] == Green:
                    self.lose_win_length_label.config(text=f"You win! The word was {self.actualWord}")
                    self.btnSubmit.config(state=DISABLED)
                elif self.turn >= 5:
                    self.lose_win_length_label.config(text=f"You Lose! The word was {self.actualWord}")
                    self.btnSubmit.config(state=DISABLED)

                self.turn += 1
                self.ClearText()

    def end_game(self):
        self.on_window_close()  

    def restart_game(self):
        self.root.destroy()  
        game_instance = WordleGame()  

class TimerApp:
    def __init__(self, root, game_instance, difficulty):
        self.root = root
        self.game_instance = game_instance
        self.time_var = tk.StringVar()

        if difficulty == "Easy":
            self.seconds = 180  
        elif difficulty == "Medium":
            self.seconds = 120  
        elif difficulty == "Hard":
            self.seconds = 60   
        else:
            raise ValueError("Invalid difficulty level")

        self.update_display()

        self.label = tk.Label(root, textvariable=self.time_var, font=("Arial", 18))
        self.label.pack(pady=10)

        self.start_timer()

    def start_timer(self):
        self.update_timer()

    def update_timer(self):
        if self.seconds > 0:
            self.seconds -= 1
            self.update_display()
            self.root.after(1000, self.update_timer)
        else:
            self.game_instance.end_game()

    def update_display(self):
        minutes, seconds = divmod(self.seconds, 60)
        time_str = f"{minutes:02d}:{seconds:02d}"
        self.time_var.set(time_str)

    def stop_timer(self):
        self.root.destroy()  

if __name__ == "__main__":
    game_instance = WordleGame()


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\anaconda\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Dell\AppData\Local\Temp\ipykernel_33328\3247512488.py", line 131, in on_window_close
    self.root.destroy()
  File "D:\anaconda\lib\tkinter\__init__.py", line 2312, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed
